In [1]:
%load_ext autoreload
%autoreload

In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import logging
logging.getLogger().setLevel(logging.INFO)

import copy
import multiprocessing as mp

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash

from config import config
from pof.loader.asset_model_loader import AssetModelLoader
from pof import Component, FailureMode, Task
from pof.interface.dashlogger import DashLogger
from pof.interface.layouts import *
from pof.interface.figures import update_condition_fig, update_cost_fig, update_pof_fig, make_inspection_interval_fig,make_sensitivity_fig

In [2]:
comp = Component.demo()
fm = FailureMode.demo()

In [3]:
# # Turn off logging
# logging.getLogger().setLevel(logging.INFO)
# #config['Load']['on_error_use_default'] ='False'

# filename = os.path.dirname(os.getcwd()) + r"\data\inputs\Asset Model - Demo.xlsx"

# aml = AssetModelLoader(filename)
# comp_data = aml.load()
# comp = Component.from_dict(comp_data['pole'])

In [4]:
comp.mc_timeline(10000)
update_cost_fig(comp)

100%|██████████| 100/100 [00:07<00:00, 12.86it/s]


In [5]:
comp.mc_timeline(1000)
update_condition_fig(comp)

100%|██████████| 100/100 [00:02<00:00, 34.42it/s]


In [6]:
df = comp.get_update_ids()
df[5]

'comp-fm-early_life-dists-untreated-alpha'

In [7]:
dropdown_list = []
for value in comp.get_update_ids():
    dropdown_dict = {}
    dropdown_dict["label"] = value
    dropdown_dict["value"] = value
    dropdown_list.append(dropdown_dict)


In [8]:
make_sensitivity_fig(comp, var_name='comp-fm-early_life-dists-untreated-alpha', t_min=0, t_max=50, step_size=10, n_iterations=10)

100%|██████████| 10/10 [00:00<00:00, 113.63it/s]


In [9]:
comp.mc_timeline(10000)
update_pof_fig(comp)

100%|██████████| 100/100 [00:01<00:00, 52.83it/s]
